In [ ]:
# Initialization cell
try:  # for CS1302 JupyterLite pyodide kernel
    import piplite

    with open("requirements.txt") as f:
        for package in f:
            package = package.strip()
            print("Installing", package)
            await piplite.install(package)
except ModuleNotFoundError:
    pass

import jupytext
import otter

grader = otter.Notebook("main.ipynb")

# Lab 3b

**CS1302 Introduction to Computer Programming**
___

In [ ]:
from math import floor, log2
from inspect import getsource
import numpy as np
from ipywidgets import interact
%reload_ext divewidgets

In this notebook, we will use iterations to convert numbers with arbitrary size.

## Conversion to Decimal

### Binary-to-Decimal

In a previous lab, we considered converting a byte string to decimal. What about converting a binary string of arbitrary length to decimal?

The simplest way is to use `int`:

In [ ]:
int("0" * 4 + "1" * 4, base=2)

**But how is `int` implemented?**

---

**Definition** (Binary numbers)

Given a binary string of an arbitrarily length $k$,

$$ 
b_{k-1}\circ \dots \circ b_1\circ b_0,
$$
the decimal number is given by the formula

$$
2^0 \cdot b_0 + 2^1 \cdot b_1 + \dots + 2^{k-1} \cdot b_{k-1}.
$$ (b2d:1)

---

In mathematics, we use the [summation](https://en.wikipedia.org/wiki/Summation) notation to write the above formula {eq}`b2d:1` concisely as

$$ 
\sum_{i=0}^{k-1} 2^i \cdot b_i,
$$ (b2d:2)

i.e., the sum of $2^i \cdot b_i$ for $i$ taking an integer value from $0$ to $k-1$.

This can be implemented using a `for` loop:

In [ ]:
def binary_to_decimal_v1(binary_str):
    k = len(binary_str)
    decimal = 0  # initialization
    for i in range(k):  # iteration
        decimal += 2**i * (1 if binary_str[(k - 1) - i] == "1" else 0)
    return decimal


binary_to_decimal_v1("0" * 4 + "1" * 4)

````{attention}
In the above implementation, 

```python
1 if binary_str[(k - 1) - i] == "1" else 0
```

is to avoid using `int` as in `int(binary_str[(k - 1) - i])`.
````

Note that $b_i$ is given by `binary_str[(k-1)-i]` for different index `i` as shown below:

$$
\begin{array}{c|c:c:c:c|}
\texttt{binary\_str} & b_{k-1} & b_{k-2} & \dots & b_0\\ \text{indexing} & [0] & [1] & \dots & [k-1] 
\end{array}
$$

The following is another way to write the `for` loop.

In [ ]:
def binary_to_decimal_v2(binary_str):
    decimal = 0  # initialization
    for bit in binary_str:
        decimal = decimal * 2 + (1 if bit == "1" else 0)  # iteration
    return decimal


binary_to_decimal_v2("0" * 4 + "1" * 4)

The algorithm implements the same formula factorized as follows:

$$
\begin{aligned} \sum_{i=0}^{k-1} 2^i \cdot b_{i} 
&=  \left(\sum_{i=1}^{k-1} 2^i \cdot b_{i}\right) + b_0\\
&=  \left(\sum_{i=1}^{k-1} 2^{i-1} \cdot b_{i}\right)\times 2 + b_0 \\
&=  \left(\sum_{j=0}^{k-2} 2^{j} \cdot b_{j+1}\right)\times 2 + b_0 && \text{with $j=i-1$} \\
&= \underbrace{(\dots (\underbrace{(\underbrace{\overbrace{0}^{\text{initialization}\kern-2em}\times 2 + b_{k-1}}_{\text{first iteration} }) \times 2 + b_{k-2}}_{\text{second iteration} }) \dots )\times 2 + b_0}_{\text{last iteration} }.\end{aligned}
$$ (b2d:3)

**Exercise** (efficiency)

Explain whether `binary_to_decimal_v1` or `binary_to_decimal_v2` runs faster?

_Type your answer here, replacing this text._

**Exercise** (binary-to-decimal) 

Write your own converter `binary_to_decimal` below using a `while` loop instead of a `for` loop. 

```{important}

Your code should contain `while` but NOT `for` nor `int`.
```

In [ ]:
def binary_to_decimal(binary_str):
    k = len(binary_str)
    decimal = 0
    ...
    return decimal

In [ ]:
# test validity
assert getsource(binary_to_decimal).find("for") < 0  # no for
assert getsource(binary_to_decimal).find("int") < 0  # no int
assert getsource(binary_to_decimal).find("while") > 0  # use while

In [ ]:
grader.check("binary-to-decimal")

In [ ]:
# binary-to-decimal converter
from ipywidgets import interact

bits = ["0", "1"]


@interact(binary_str="1011")
def convert_byte_to_decimal(binary_str):
    for bit in binary_str:
        if bit not in bits:
            print("Not a binary string.")
            break
    else:
        print("decimal:", binary_to_decimal(binary_str))

### Undecimal-to-Decimal

A base-11 number system is called an [undecimal system](https://en.wikipedia.org/wiki/Undecimal). The digits range from 0 to 10 with 10 denoted as X:

$$
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, X.
$$

The [International Standard Book Number (ISBN)](https://en.wikipedia.org/wiki/International_Standard_Book_Number) uses an undecimal digit.

**Exercise** (undecimal-to-decimal)

In the following code, assign to `decimal` the integer represented by an undecimal string of arbitrary length.

```{important}

Your code should NOT contain `int`. You may assume `undecimal_str` is valid undecimal string consisting of characters from "0123456789X".
```

In [ ]:
def undecimal_to_decimal(undecimal_str):
    ...
    return decimal

In [ ]:
# test validity
assert getsource(undecimal_to_decimal).find("int") < 0  # no int

In [ ]:
grader.check("undecimal-to-decimal")

In [ ]:
# undecimal-to-decimal calculator
undecimal_digits = [str(i) for i in range(10)] + ["X"]

@interact(undecimal_str="X")
def convert_undecimal_to_decimal(undecimal_str):
    for digit in undecimal_str:
        if digit not in undecimal_digits:
            print("Not an undecimal string.")
            break
    else:
        print("decimal:", undecimal_to_decimal(undecimal_str))

## Conversion from Decimal

Consider the reverse process that converts a non-negative decimal number of arbitrary size to a string representation in another number system.

### Decimal-to-Binary

The following code converts a decimal number to a binary string.

```Python
def decimal_to_binary(decimal):
    binary_str = str(decimal % 2)
    while decimal // 2:
        decimal //= 2
        binary_str = str(decimal % 2) + binary_str
    return binary_str
```

To understand the while loop, consider the same formula {eq}`b2d:3`, where the braces indicate the value of `decimal` at different times:

$$
\begin{aligned} \sum_{i=0}^{k-1} 2^i \cdot b_{i} &=  \left(\sum_{i=0}^{k-2} 2^{i-2} \cdot b_{i-1}\right)\times 2 + b_0 \\
&= \underbrace{(\underbrace{ \dots (\underbrace{(0\times 2 + b_{k-1}) \times 2 + b_{k-2}}_{\text{right before the last iteration} }  )\times 2 \dots + b_1}_{\text{right before the second iteration} })\times 2 + b_0}_{\text{right before the first iteration} }.\end{aligned}
$$ 

- $b_0$ is the remainder `decimal % 2` right before the first iteration,
- $b_1$ is the remainder `decimal // 2 % 2` right before the second iteration, and
- $b_{k-1}$ is the remainder `decimal // 2 % 2` right before the last iteration.

We can also write a for loop instead of a while loop:

In [ ]:
def decimal_to_binary(decimal):
    binary_str = ""
    num_bits = 1 + (decimal and floor(log2(decimal)))
    for i in range(num_bits):
        binary_str = str(decimal % 2) + binary_str
        decimal //= 2
    return binary_str

In [ ]:
# decimal-to-binary calculator
@interact(decimal="11")
def convert_decimal_to_binary(decimal):
    if not decimal.isdigit():
        print("Not a non-negative integer.")
    else:
        print("binary:", decimal_to_binary(int(decimal)))

**Exercise** (logical-and)

Explain what the expression `1 + (decimal and floor(log2(decimal)))` calculates. In particular, explain the purpose of the logical `and` operation in the expression?

```{hint}
`floor(log2(decimal))` computes $\lfloor \log_2 (\texttt{decimal})\rfloor$, namely, the smallest integer no larger than the logarithm base 2 of some number `decimal`.
- What happen when you run `floor(log2(decimal))` when `decimal` equals `0`?
- How about `(decimal and floor(log2(decimal))` when `decimal` equals `0`? Apply the short-circuit evaluation rule.
- What does `1 + (decimal and floor(log2(decimal)))` return when `decimal` is between $2^{\ell} \leq \texttt{decimal} < 2^{\ell + 1}$?
```

_Type your answer here, replacing this text._

### Decimal-to-Undecimal

**Exercise** (decimal-to-undecimal)

Assign to `undecimal_str` the undecimal string that represents a non-negative integer `decimal` of any size.

In [ ]:
def decimal_to_undecimal(decimal):
    ...
    return undecimal_str

In [ ]:
grader.check("decimal-to-undecimal")

In [ ]:
# undecimal-to-decimal calculator
@interact(decimal="10")
def convert_decimal_to_undecimal(decimal):
    if not decimal.isdigit():
        print("Not a non-negative integer.")
    else:
        print("undecimal:", decimal_to_undecimal(int(decimal)))

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# extra files to submit
extra_files = []

In [ ]:
# Generate the source main.py necessary for grading and similarity check.
jupytext.write(jupytext.read("main.ipynb"), "main.py", fmt="py:percent")

In [ ]:
# Generate the zip file to submit.
grader.export(pdf=False, run_tests=False, files=["main.py", *extra_files])